In [5]:
from datetime import datetime
import urllib.request, json
import ttn


class ttn_handler:
    def __init__(self, app_id, access_key, callback):
        self.handler = ttn.HandlerClient(app_id, access_key)
        self.mqtt_client = self.handler.data()
        self.mqtt_client.set_uplink_callback(callback)
        self.mqtt_client.connect()
        self.location_cache = dict()
        
    def __del__(self):
        self.mqtt_client.close()
        
    def gtw_location(self, gtw_id):
        if gtw_id in self.location_cache:
            return self.location_cache[gtw_id]
        
        try:
            with urllib.request.urlopen('https://www.thethingsnetwork.org/gateway-data/gateway/' + gtw_id) as url:
                data = json.loads(url.read().decode())
                loc_raw = data[gtw_id]['location']
                loc = (loc_raw['latitude'], loc_raw['longitude'])
                self.location_cache[gtw_id] = loc
                return loc
        except:
            return None
        
    def rssi_to_dist(self, rssi):
        return int(10.0 ** ((12-rssi)/(4*10)))
    
    def msg_timestamp(msg):
        timestring = msg['metadata']['time'].split('.', 1)[0]
        return int(datetime.strptime(timestring, '%Y-%m-%dT%H:%M:%S').timestamp())

    # TTN namedtuples are annoying to pickle...
    def msg_to_dict(msg):
        retval = msg._asdict()
        retval['metadata'] = retval['metadata']._asdict()
        retval['metadata']['gateways'] = [e._asdict() for e in retval['metadata']['gateways']]
        return retval